In [1]:
import numpy as np
import pandas as pd

In [2]:
lab_dict = pd.read_csv('D_LABITEMS.csv', sep = ',')
labevents = pd.read_csv('LABEVENTS.csv', sep = ',')

In [3]:
labevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [24]:
group = labevents.fillna({'VALUEUOM':'?'}).groupby(['ITEMID', 'VALUEUOM'])
lab_sum = group.size().to_frame(name = 'counts')
labevents.set_index('ROW_ID')
lab_sum = (lab_sum
    .join(group.agg({'VALUENUM':'mean'}).rename(columns={'VALUENUM':'mean'}))
    .join(group.agg({'VALUENUM':'median'}).rename(columns={'VALUENUM':'median'}))
    .reset_index()
)
lab_sum.head()

,ITEMID,VALUEUOM,counts,mean,median
0,50800,?,404785,NaN,NaN
1,50801,?,5943,457.646524,475.0
2,50801,mm Hg,16073,478.099981,501.0
3,50802,?,20,5.100000,5.0
4,50802,mEq/L,490631,-0.090816,0.0


In [25]:
(labevents[labevents.VALUENUM.notnull()]
 .loc[:,['SUBJECT_ID', 'ITEMID', 'VALUENUM', 'VALUEUOM', 'FLAG']]
 .head()
)

,SUBJECT_ID,ITEMID,VALUENUM,VALUEUOM,FLAG
0,3,50820,7.39,units,NaN
2,3,50802,-1.00,mEq/L,NaN
3,3,50804,22.00,mEq/L,NaN
4,3,50808,0.93,mmol/L,abnormal
6,3,50813,1.80,mmol/L,NaN


In [29]:
lab_dict.head()

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


In [35]:
lab_sum.merge(lab_dict, on = 'ITEMID', how = 'outer').head()

,ITEMID,VALUEUOM,counts,mean,median,ROW_ID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,50800,?,404785.0,NaN,NaN,1,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,50801,?,5943.0,457.646524,475.0,2,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,50801,mm Hg,16073.0,478.099981,501.0,2,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
3,50802,?,20.0,5.100000,5.0,3,Base Excess,Blood,Blood Gas,11555-0
4,50802,mEq/L,490631.0,-0.090816,0.0,3,Base Excess,Blood,Blood Gas,11555-0


In [4]:
lab_dict.head()

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


In [5]:
mimic2hpo = pd.read_csv('~/Desktop/mimic2hpo.tsv', sep = '\t', header = None)

In [6]:
mimic2hpo.rename(columns = {0:'row_id', 
                            1:'item_id', 
                            2:'value', 
                            3:'flag', 
                            4:'negated',
                            5:'hpo'}, inplace = True)
mimic2hpo.head()

,row_id,item_id,value,flag,negated,hpo
0,281,50820,7.39,NaN,T,HP:0004360
1,282,50800,ART,NaN,U,ERROR 1: local id not mapped to loinc
2,283,50802,-1,NaN,T,HP:0032281
3,284,50804,22,NaN,T,HP:0500164
4,285,50808,0.93,abnormal,F,HP:0002901


In [ ]:
# Show the percentage of success rate and different kinds of errors

In [9]:
pd.DataFrame({'percentage': 
              mimic2hpo.assign(cat = ['HPO' if x.startswith('HP') else x for x in mimic2hpo.hpo])
              .groupby('cat').size() / len(mimic2hpo) })


,percentage
cat,
ERROR 1: local id not mapped to loinc,0.035987
ERROR 3: loinc code not annotated,0.068701
ERROR 4: interpretation code not mapped to hpo,0.000112
ERROR 5: unable to interpret,0.024710
HPO,0.870490


In [18]:
mimic2hpo.set_index(mimic2hpo.row_id)
labevents.set_index(labevents.ROW_ID)

In [23]:
combined = labevents.join(mimic2hpo[['negated', 'hpo']], how = 'left')
combined.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,negated,hpo
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN,T,HP:0004360
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN,U,ERROR 1: local id not mapped to loinc
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN,T,HP:0032281
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN,T,HP:0500164
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal,F,HP:0002901


In [47]:
total_hp_count_per_patient = combined[combined.hpo.str.startswith('HP')].groupby('SUBJECT_ID').size().reset_index()
total_hp_count_per_patient.rename(columns = {'SUBJECT_ID': 'patient', 0: 'hpo_n'}, inplace = True)

In [58]:
total_hp_count_per_patient.head()

,patient,hpo_n
0,2,40
1,3,1303
2,4,1433
3,5,17
4,6,1065


In [80]:
bins = pd.cut(total_hp_count_per_patient.hpo_n, bins = [-1, 0, 100, 500, 1500, 4000, 30000])

In [81]:
total_hp_count_per_patient.groupby(bins).size()

hpo_n
(-1, 0]              0
(0, 100]         10247
(100, 500]       22261
(500, 1500]      10404
(1500, 4000]      2876
(4000, 30000]      463
dtype: int64

In [88]:
abnormal_hp_count_per_patient = combined[combined.negated == 'F'].groupby('SUBJECT_ID').size().reset_index()


#.groupby(pd.cut('0', bins = [0, 100, 500, 1000, 30000])).size()

In [100]:
abnormal_hp_count_per_patient.groupby(pd.cut(abnormal_hp_count_per_patient.loc[:,0], bins = [0, 100, 500, 1000, 5000, 30000])).size()

0
(0, 100]         24032
(100, 500]       17490
(500, 1000]       2912
(1000, 5000]      1466
(5000, 30000]       17
dtype: int64

In [101]:
patient_group = combined[combined.negated == 'F'].groupby('SUBJECT_ID')

In [108]:
for group_id, group in patient_group:
    print('patient id: {}, unique abnormal hp terms: {}'.format( group_id, len(group.hpo.unique())))

patient id: 2, unique abnormal hp terms: 14
patient id: 3, unique abnormal hp terms: 42
patient id: 4, unique abnormal hp terms: 66
patient id: 5, unique abnormal hp terms: 5
patient id: 6, unique abnormal hp terms: 54
patient id: 7, unique abnormal hp terms: 10
patient id: 8, unique abnormal hp terms: 11
patient id: 9, unique abnormal hp terms: 35
patient id: 10, unique abnormal hp terms: 8
patient id: 11, unique abnormal hp terms: 23
patient id: 12, unique abnormal hp terms: 55
patient id: 13, unique abnormal hp terms: 34
patient id: 16, unique abnormal hp terms: 8
patient id: 17, unique abnormal hp terms: 38
patient id: 18, unique abnormal hp terms: 20
patient id: 19, unique abnormal hp terms: 21
patient id: 20, unique abnormal hp terms: 21
patient id: 21, unique abnormal hp terms: 56
patient id: 22, unique abnormal hp terms: 4
patient id: 23, unique abnormal hp terms: 31
patient id: 24, unique abnormal hp terms: 9
patient id: 25, unique abnormal hp terms: 36
patient id: 26, unique 

patient id: 260, unique abnormal hp terms: 5
patient id: 261, unique abnormal hp terms: 26
patient id: 262, unique abnormal hp terms: 18
patient id: 263, unique abnormal hp terms: 25
patient id: 264, unique abnormal hp terms: 7
patient id: 265, unique abnormal hp terms: 32
patient id: 266, unique abnormal hp terms: 27
patient id: 267, unique abnormal hp terms: 42
patient id: 268, unique abnormal hp terms: 41
patient id: 269, unique abnormal hp terms: 37
patient id: 270, unique abnormal hp terms: 14
patient id: 271, unique abnormal hp terms: 31
patient id: 272, unique abnormal hp terms: 14
patient id: 273, unique abnormal hp terms: 15
patient id: 274, unique abnormal hp terms: 40
patient id: 275, unique abnormal hp terms: 36
patient id: 276, unique abnormal hp terms: 15
patient id: 279, unique abnormal hp terms: 43
patient id: 280, unique abnormal hp terms: 13
patient id: 281, unique abnormal hp terms: 41
patient id: 282, unique abnormal hp terms: 37
patient id: 283, unique abnormal hp 

patient id: 1019, unique abnormal hp terms: 24
patient id: 1020, unique abnormal hp terms: 38
patient id: 1021, unique abnormal hp terms: 19
patient id: 1022, unique abnormal hp terms: 5
patient id: 1023, unique abnormal hp terms: 8
patient id: 1024, unique abnormal hp terms: 52
patient id: 1025, unique abnormal hp terms: 5
patient id: 1026, unique abnormal hp terms: 27
patient id: 1027, unique abnormal hp terms: 44
patient id: 1028, unique abnormal hp terms: 40
patient id: 1029, unique abnormal hp terms: 34
patient id: 1030, unique abnormal hp terms: 41
patient id: 1031, unique abnormal hp terms: 9
patient id: 1032, unique abnormal hp terms: 24
patient id: 1033, unique abnormal hp terms: 36
patient id: 1034, unique abnormal hp terms: 27
patient id: 1035, unique abnormal hp terms: 8
patient id: 1036, unique abnormal hp terms: 60
patient id: 1037, unique abnormal hp terms: 53
patient id: 1038, unique abnormal hp terms: 34
patient id: 1039, unique abnormal hp terms: 33
patient id: 1040, 

patient id: 2121, unique abnormal hp terms: 45
patient id: 2122, unique abnormal hp terms: 9
patient id: 2123, unique abnormal hp terms: 31
patient id: 2124, unique abnormal hp terms: 70
patient id: 2125, unique abnormal hp terms: 9
patient id: 2127, unique abnormal hp terms: 37
patient id: 2128, unique abnormal hp terms: 20
patient id: 2129, unique abnormal hp terms: 7
patient id: 2130, unique abnormal hp terms: 33
patient id: 2131, unique abnormal hp terms: 9
patient id: 2132, unique abnormal hp terms: 19
patient id: 2133, unique abnormal hp terms: 16
patient id: 2134, unique abnormal hp terms: 12
patient id: 2135, unique abnormal hp terms: 42
patient id: 2136, unique abnormal hp terms: 60
patient id: 2137, unique abnormal hp terms: 5
patient id: 2138, unique abnormal hp terms: 7
patient id: 2139, unique abnormal hp terms: 24
patient id: 2140, unique abnormal hp terms: 6
patient id: 2141, unique abnormal hp terms: 10
patient id: 2142, unique abnormal hp terms: 7
patient id: 2143, uni

patient id: 3216, unique abnormal hp terms: 55
patient id: 3217, unique abnormal hp terms: 41
patient id: 3218, unique abnormal hp terms: 44
patient id: 3219, unique abnormal hp terms: 6
patient id: 3220, unique abnormal hp terms: 30
patient id: 3221, unique abnormal hp terms: 22
patient id: 3222, unique abnormal hp terms: 40
patient id: 3223, unique abnormal hp terms: 10
patient id: 3224, unique abnormal hp terms: 6
patient id: 3225, unique abnormal hp terms: 54
patient id: 3226, unique abnormal hp terms: 29
patient id: 3227, unique abnormal hp terms: 29
patient id: 3228, unique abnormal hp terms: 9
patient id: 3229, unique abnormal hp terms: 7
patient id: 3231, unique abnormal hp terms: 24
patient id: 3232, unique abnormal hp terms: 46
patient id: 3233, unique abnormal hp terms: 8
patient id: 3234, unique abnormal hp terms: 28
patient id: 3235, unique abnormal hp terms: 50
patient id: 3236, unique abnormal hp terms: 36
patient id: 3237, unique abnormal hp terms: 7
patient id: 3238, u

...

patient id: 81737, unique abnormal hp terms: 30
patient id: 81740, unique abnormal hp terms: 19
patient id: 81745, unique abnormal hp terms: 34
patient id: 81750, unique abnormal hp terms: 32
patient id: 81754, unique abnormal hp terms: 33
patient id: 81755, unique abnormal hp terms: 26
patient id: 81758, unique abnormal hp terms: 28
patient id: 81763, unique abnormal hp terms: 58
patient id: 81766, unique abnormal hp terms: 14
patient id: 81770, unique abnormal hp terms: 22
patient id: 81773, unique abnormal hp terms: 36
patient id: 81775, unique abnormal hp terms: 21
patient id: 81778, unique abnormal hp terms: 32
patient id: 81783, unique abnormal hp terms: 30
patient id: 81786, unique abnormal hp terms: 60
patient id: 81787, unique abnormal hp terms: 14
patient id: 81791, unique abnormal hp terms: 28
patient id: 81793, unique abnormal hp terms: 25
patient id: 81797, unique abnormal hp terms: 38
patient id: 81807, unique abnormal hp terms: 36
patient id: 81808, unique abnormal hp te

patient id: 85644, unique abnormal hp terms: 25
patient id: 85645, unique abnormal hp terms: 14
patient id: 85647, unique abnormal hp terms: 31
patient id: 85649, unique abnormal hp terms: 13
patient id: 85650, unique abnormal hp terms: 27
patient id: 85655, unique abnormal hp terms: 37
patient id: 85658, unique abnormal hp terms: 23
patient id: 85659, unique abnormal hp terms: 37
patient id: 85665, unique abnormal hp terms: 10
patient id: 85669, unique abnormal hp terms: 33
patient id: 85672, unique abnormal hp terms: 26
patient id: 85673, unique abnormal hp terms: 22
patient id: 85685, unique abnormal hp terms: 46
patient id: 85688, unique abnormal hp terms: 25
patient id: 85689, unique abnormal hp terms: 19
patient id: 85690, unique abnormal hp terms: 12
patient id: 85698, unique abnormal hp terms: 27
patient id: 85699, unique abnormal hp terms: 73
patient id: 85700, unique abnormal hp terms: 39
patient id: 85704, unique abnormal hp terms: 31
patient id: 85711, unique abnormal hp te

patient id: 89329, unique abnormal hp terms: 44
patient id: 89332, unique abnormal hp terms: 20
patient id: 89334, unique abnormal hp terms: 16
patient id: 89336, unique abnormal hp terms: 21
patient id: 89345, unique abnormal hp terms: 10
patient id: 89347, unique abnormal hp terms: 35
patient id: 89348, unique abnormal hp terms: 28
patient id: 89355, unique abnormal hp terms: 38
patient id: 89356, unique abnormal hp terms: 66
patient id: 89363, unique abnormal hp terms: 28
patient id: 89368, unique abnormal hp terms: 23
patient id: 89369, unique abnormal hp terms: 20
patient id: 89374, unique abnormal hp terms: 9
patient id: 89378, unique abnormal hp terms: 38
patient id: 89379, unique abnormal hp terms: 40
patient id: 89386, unique abnormal hp terms: 21
patient id: 89387, unique abnormal hp terms: 6
patient id: 89394, unique abnormal hp terms: 24
patient id: 89398, unique abnormal hp terms: 26
patient id: 89400, unique abnormal hp terms: 10
patient id: 89402, unique abnormal hp term

patient id: 93219, unique abnormal hp terms: 18
patient id: 93222, unique abnormal hp terms: 28
patient id: 93227, unique abnormal hp terms: 47
patient id: 93229, unique abnormal hp terms: 25
patient id: 93231, unique abnormal hp terms: 22
patient id: 93233, unique abnormal hp terms: 35
patient id: 93235, unique abnormal hp terms: 24
patient id: 93240, unique abnormal hp terms: 26
patient id: 93245, unique abnormal hp terms: 11
patient id: 93261, unique abnormal hp terms: 21
patient id: 93262, unique abnormal hp terms: 34
patient id: 93265, unique abnormal hp terms: 12
patient id: 93272, unique abnormal hp terms: 31
patient id: 93277, unique abnormal hp terms: 26
patient id: 93279, unique abnormal hp terms: 14
patient id: 93280, unique abnormal hp terms: 22
patient id: 93284, unique abnormal hp terms: 33
patient id: 93290, unique abnormal hp terms: 30
patient id: 93299, unique abnormal hp terms: 28
patient id: 93301, unique abnormal hp terms: 20
patient id: 93307, unique abnormal hp te

patient id: 96814, unique abnormal hp terms: 11
patient id: 96815, unique abnormal hp terms: 25
patient id: 96817, unique abnormal hp terms: 20
patient id: 96821, unique abnormal hp terms: 11
patient id: 96825, unique abnormal hp terms: 50
patient id: 96833, unique abnormal hp terms: 43
patient id: 96838, unique abnormal hp terms: 21
patient id: 96842, unique abnormal hp terms: 34
patient id: 96843, unique abnormal hp terms: 26
patient id: 96848, unique abnormal hp terms: 46
patient id: 96863, unique abnormal hp terms: 20
patient id: 96864, unique abnormal hp terms: 35
patient id: 96865, unique abnormal hp terms: 30
patient id: 96869, unique abnormal hp terms: 49
patient id: 96872, unique abnormal hp terms: 34
patient id: 96873, unique abnormal hp terms: 29
patient id: 96877, unique abnormal hp terms: 14
patient id: 96879, unique abnormal hp terms: 44
patient id: 96881, unique abnormal hp terms: 21
patient id: 96882, unique abnormal hp terms: 24
patient id: 96888, unique abnormal hp te

In [125]:
unique_abnormal_hp_count = pd.DataFrame({'patient_id': [group_id for group_id, _ in patient_group],
                                        'abnormal_hpo_n': [len(group.hpo.unique()) for _, group in patient_group]})

In [129]:
unique_abnormal_hp_count.set_index('patient_id')
unique_abnormal_hp_count.abnormal_hpo_n.median()

26.0

In [132]:
labevents.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [ ]:
combined.groupby([patient_id])